## Hyper-parameter tuning

In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import numpy as np
import pandas as pd
import torch


In [4]:
from src.models import FCN_with_last_activation
from src.constants import TIMESTEP_SCALE, GAMMA_SCALE, N_SCALE
from src.physics_loss import physics_loss_fixed_gamma_n, physics_loss_varied_gamma_n
from src.cross_validation import cross_validation

In [5]:
t_steps = 10000

# define the Universe time
u_time = np.arange(t_steps) # *dt.value

In [6]:
seed = 123
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

In [7]:
data = np.load('data/test1.npy').T
data.shape

(2000, 10000)

In [8]:
params = pd.read_csv('data/params1.txt', sep=' ')
params = params.values
params.shape

(2000, 2)

In [10]:
timesteps = torch.tensor(u_time).float().view(-1,1)
y = torch.tensor(data).float()

In [11]:
x = torch.zeros((2000, 10000, 3)) # n_simulations * n_timesteps * feature_dim
for i, row in enumerate(params):
    row = row * np.array([N_SCALE, GAMMA_SCALE])
    row_2d = np.broadcast_to(row, (10000, 2))
    row_2d = torch.tensor(row_2d).float()
    x[i] = torch.hstack((timesteps * TIMESTEP_SCALE, row_2d))
print(x[0, 0:5, :])

tensor([[0.0000e+00, 1.6522e-02, 1.6896e+00],
        [1.0000e-04, 1.6522e-02, 1.6896e+00],
        [2.0000e-04, 1.6522e-02, 1.6896e+00],
        [3.0000e-04, 1.6522e-02, 1.6896e+00],
        [4.0000e-04, 1.6522e-02, 1.6896e+00]])


# Model for one set of parameters  

In [12]:
x_time = x[0, :, 0].view(-1, 1)
y_one_params = y[0].view(-1, 1)
gamma, nH = params[0].astype(float)

In [13]:
grid_physics_coef = [1e3, 1e-4, 1e4]
best_score = np.Inf
physics_loss = lambda x, y, loss_coef: physics_loss_fixed_gamma_n(x, y, gamma, nH, loss_coef)
for physics_coef in grid_physics_coef:
    r2_score = cross_validation(model_class=FCN_with_last_activation, model_args={"module_dims": [3, 8, 8, 8, 8, 1], "last_activation": torch.nn.Tanh}, x=x_time, y=y_one_params, iters=5, physics_loss=physics_loss, 
                                physics_coef=physics_coef)
    if r2_score > best_score:
            best_score = r2_score
            best_physics_coef = physics_coef
print(f"The best r2 score is {best_score:.3f} with a physics-loss-coefficient of {best_physics_coef}")


torch.Size([100, 1]) torch.Size([9900, 1])


  0%|          | 0/5 [00:00<?, ?it/s]


RuntimeError: mat1 and mat2 shapes cannot be multiplied (100x1 and 3x8)

# Model for multiple simulations

In [15]:
x = torch.zeros((2000, 10000, 3)) # n_simulations * n_timesteps * feature_dim
for i, row in enumerate(params):
    row = row * np.array([N_SCALE, GAMMA_SCALE])
    row_2d = np.broadcast_to(row, (10000, 2))
    row_2d = torch.tensor(row_2d).float()
    x[i] = torch.hstack((timesteps * TIMESTEP_SCALE, row_2d))
timesteps = torch.tensor(u_time).float().view(-1,1)
y = torch.tensor(data).float()

In [16]:
grid_physics_coef = [1e3, 1e-4, 1e4]
best_score = np.Inf
physics_loss = lambda x, y, loss_coef: physics_loss_varied_gamma_n(x, y, loss_coef)
for physics_coef in grid_physics_coef:
    r2_score = cross_validation(model_class=FCN_with_last_activation, model_args={"module_dims": [3, 8, 8, 8, 8, 1], "last_activation": torch.nn.Tanh}, x=x_time, y=y_one_params, iters=5, physics_loss=physics_loss, 
                                physics_coef=physics_coef)
    if r2_score > best_score:
            best_score = r2_score
            best_physics_coef = physics_coef
print(f"The best r2 score is {best_score:.3f} with a physics-loss-coefficient of {best_physics_coef}")

torch.Size([100, 1]) torch.Size([9900, 1])


  0%|          | 0/5 [00:00<?, ?it/s]


RuntimeError: mat1 and mat2 shapes cannot be multiplied (100x1 and 3x8)